Importing Modules

In [1]:
import pandas as pd
import seaborn as sns
%matplotlib inline 

ModuleNotFoundError: No module named 'pandas'

In [ ]:
data_url = 'data/car data.csv'

In [ ]:
df = pd.read_csv(data_url)
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

Checking Categorical Values


In [ ]:
# checking features
cat = df.select_dtypes(include='O').keys()
# display variabels
cat

In [ ]:
# creating new df
# setting columns we use
new_df = pd.read_csv(data_url,usecols=['Car_Name', 'Fuel_Type', 'Seller_Type', 'Transmission']) 
new_df.head()

In [ ]:
# unique values in each columns
for x in new_df.columns:
    #prinfting unique values
    print(x ,':', len(new_df[x].unique()))

In [ ]:
# finding the top 20 categories
new_df.Car_Name.value_counts().sort_values(ascending=False).head(20)

In [ ]:
sns.countplot(x='Fuel_Type',data=df)

In [ ]:
sns.barplot(x='Fuel_Type',y='Selling_Price',data=df)

In [ ]:
sns.boxplot(x='Fuel_Type',y='Selling_Price',data=df,palette='rainbow')

In [ ]:
sns.boxplot(data=df,orient='h')

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df_group = df.groupby(['Car_Name'])['Selling_Price'].mean()
df_group.plot.pie(figsize=(10,20), autopct="%.2f")

In [ ]:
final_dataset=df[['Year','Selling_Price','Present_Price','Kms_Driven','Fuel_Type','Seller_Type','Transmission','Owner']]

In [ ]:
final_dataset.head()


In [ ]:
final_dataset['Current Year']=2020
final_dataset.head()


In [ ]:
final_dataset['no_year']=final_dataset['Current Year']- final_dataset['Year']
final_dataset.head()


In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)


In [ ]:
final_dataset=final_dataset.drop(['Current Year'],axis=1)


In [ ]:
final_dataset=pd.get_dummies(final_dataset,drop_first=True)


In [ ]:
final_dataset.corr()


In [ ]:
import seaborn as sns
sns.pairplot(final_dataset)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
X=final_dataset.iloc[:,1:]
y=final_dataset.iloc[:,0]

In [ ]:
X['Owner'].unique()

In [ ]:
X.head()

In [ ]:
y.head()

Feature Importance

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [ ]:
 #Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_


In [ ]:
predictions=rf_random.predict(X_test)
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)